# Setup

In [1]:
import subprocess
import tempfile
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time
import random

### SET CWD TO REPO ROOT
os.chdir(Path.cwd().parent)
print("CWD now:", Path.cwd())

from rtl.py_lmul import lmul
from rtl.lmul_tester import BatchLMULTester

CWD now: /workspaces/LMUL-Hardware-Acceleration
✓ Batch tester ready


# Utilities

In [2]:
def float_to_bf16(f: float) -> int:
    """Convert float32 to bfloat16 (16-bit)"""
    if np.isnan(f):
        return 0x7FC0
    if np.isinf(f):
        return 0xFF80 if f < 0 else 0x7F80
    f = np.clip(f, -3.4e38, 3.4e38)
    f32_bits = struct.unpack('>I', struct.pack('>f', np.float32(f)))[0]
    bf16_bits = (f32_bits >> 16) & 0xFFFF
    return bf16_bits

def bf16_to_float(bf16: int) -> float:
    """Convert bfloat16 to float32"""
    f32_bits = (bf16 & 0xFFFF) << 16
    return struct.unpack('>f', struct.pack('>I', f32_bits))[0]

print("✓ BF16 utilities ready")



✓ BF16 utilities ready


# Tests

In [3]:
test_data = []
for _ in range(10):
    a_float = np.random.uniform(-10, 10)
    b_float = np.random.uniform(-10, 10)
    a_bf16 = float_to_bf16(a_float)
    b_bf16 = float_to_bf16(b_float)
    test_data.append((a_bf16, b_bf16, a_float, b_float))

In [4]:
tester = BatchLMULTester()
test_pairs = [(a_bf16, b_bf16) for a_bf16, b_bf16, _, _ in test_data]
lmul_res, _ = tester.test_batch(test_pairs, verbose=True)
lmul_results = [bf16_to_float(res) for res in lmul_res]


3e3b
41ec
41b0
c1db
c147
41c7
c18f
bfdd
412e
41c6



In [5]:
lmul_results = []
pylmul_results = []
py_results = []

tester = BatchLMULTester()
test_pairs = [(a_bf16, b_bf16) for a_bf16, b_bf16, _, _ in test_data]
lmul_res, _ = tester.test_batch(test_pairs, verbose=True)
lmul_results = [bf16_to_float(res) for res in lmul_res]

for a_bf16, b_bf16, a_float, b_float in test_data:
    pylmul_res = bf16_to_float(lmul(a_bf16, b_bf16))
    py_res = a_float * b_float
    lmul_results.append(lmul_res)
    pylmul_results.append(pylmul_res)
    py_results.append(py_res)

3e3b
41ec
41b0
c1db
c147
41c7
c18f
bfdd
412e
41c6



In [6]:
def format_test_case_outputs(test_data, lmul_results, pylmul_results, py_results):
    for i, (a_bf16, b_bf16, a_float, b_float) in enumerate(test_data):
        print(f"Test Case {i + 1}:")
        print(f"  a_bf16: {a_bf16}, b_bf16: {b_bf16}")
        print(f"  a_float: {a_float}, b_float: {b_float}")
        print(f"  LMUL Result: {lmul_results[i]}")
        print(f"  PyLMUL Result: {pylmul_results[i]}")
        print(f"  Python Result: {py_results[i]}")
        print("-" * 30)

format_test_case_outputs(test_data, lmul_results, pylmul_results, py_results)

Test Case 1:
  a_bf16: 48415, b_bf16: 49308
  a_float: -0.03894312432757552, b_float: -4.89483904546238
  LMUL Result: 0.1826171875
  PyLMUL Result: 0.1826171875
  Python Result: 0.19062032551091254
------------------------------
Test Case 2:
  a_bf16: 49249, b_bf16: 49419
  a_float: -3.5276318543503997, b_float: -8.713418078912836
  LMUL Result: 29.5
  PyLMUL Result: 29.5
  Python Result: 30.737731175445585
------------------------------
Test Case 3:
  a_bf16: 16443, b_bf16: 16629
  a_float: 2.93529375816175, b_float: 7.667588907304818
  LMUL Result: 22.0
  PyLMUL Result: 22.0
  Python Result: 22.506625859762107
------------------------------
Test Case 4:
  a_bf16: 16449, b_bf16: 49434
  a_float: 3.0310407894545826, b_float: -9.667063869628205
  LMUL Result: -27.375
  PyLMUL Result: -27.375
  Python Result: -29.301264903105746
------------------------------
Test Case 5:
  a_bf16: 49409, b_bf16: 16326
  a_float: -8.084546262867958, b_float: 1.5503113628866885
  LMUL Result: -12.4375
  